In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [2]:
# Download caption annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  os.remove(annotation_zip)

# Download image files
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
  image_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip) + image_folder
  os.remove(image_zip)
else:
  PATH = os.path.abspath('.') + image_folder

13510574080/13510573713 [==============================] - 326s 0us/step


In [3]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

In [4]:
# Group all captions together having the same image ID.
image_path_to_caption = collections.defaultdict(list)
for val in annotations['annotations']:
  caption = f"<start> {val['caption']} <end>"
  image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (val['image_id'])
  image_path_to_caption[image_path].append(caption)

In [5]:
image_paths = list(image_path_to_caption.keys())
random.shuffle(image_paths)

# Select the first 6000 image_paths from the shuffled set.
# Approximately each image id has 5 captions associated with it, so that will 
# lead to 30,000 examples.
train_image_paths = image_paths[:6000]
print(len(train_image_paths))

6000


In [6]:
train_captions = []
img_name_vector = []

for image_path in train_image_paths:
  caption_list = image_path_to_caption[image_path]
  train_captions.extend(caption_list)
  img_name_vector.extend([image_path] * len(caption_list))

In [7]:
print(f'===> number of images: {len(img_name_vector)}\n===> number of captions: {len(train_captions)}')

===> number of images: 30012
===> number of captions: 30012


In [8]:
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

train_seqs = tokenizer.texts_to_sequences(train_captions)

cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [9]:
def decode_sequence(encoded_text):
  lst = []
  for i in encoded_text[0]:
    lst.append(tokenizer.index_word[i])
  return ' '.join(lst)

In [10]:
def tokenize_sequence(sequence):
  return tokenizer.texts_to_sequences(sequence)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Reshape,Dropout,LeakyReLU,Flatten,BatchNormalization,Conv2D,Conv2DTranspose
from tensorflow.keras.models import Sequential

In [12]:
generator = Sequential()
generator.add(Dense(7*7*256, use_bias=False, input_shape=(47,)))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

generator.add(Reshape((7, 7, 256)))
assert generator.output_shape == (None, 7, 7, 256) # Note: None is the batch size

generator.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
assert generator.output_shape == (None, 7, 7, 128)
generator.add(BatchNormalization())
generator.add(LeakyReLU())

generator.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
assert generator.output_shape == (None, 14, 14, 64)
generator.add(BatchNormalization())
generator.add(LeakyReLU())

generator.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
assert generator.output_shape == (None, 28, 28, 1)

In [13]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, padding="same",
                        activation=LeakyReLU(0.3),
                        input_shape=[28, 28, 1]))
discriminator.add(Dropout(0.5))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same",
                        activation=LeakyReLU(0.3)))
discriminator.add(Dropout(0.5))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation="sigmoid"))

In [14]:
GAN = Sequential([generator, discriminator])

In [15]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.trainable = False

In [16]:
GAN.compile(loss="binary_crossentropy", optimizer="adam")

In [29]:
from PIL import Image

epochs = 1
batch_size = 10000
WIDTH, HEIGHT = 28, 28

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
  for epoch in range(1,epochs + 1):
    print(f'Epoch {epoch}/{epochs}')
    batch_no = 1
    train_captions, train_images = list(), list()
    import time as t
    t1 = t.time()
    for batch_captions, batch_images in zip(cap_vector, img_name_vector):
      if len(train_captions) < batch_size and len(train_images) < batch_size:
        train_captions.append(batch_captions)
        img = Image.open(batch_images).convert('L')
        resized_img = img.resize((WIDTH, HEIGHT))
        normalized_img = np.array(resized_img) / 255
        train_images.append(normalized_img.reshape(WIDTH, HEIGHT,1))

      else:
        captions = np.array(train_captions)
        print(captions.shape)
        print(f'bacth number : {batch_no}')

        gen_images = generator(captions)

        X_fake_vs_real = tf.concat([gen_images, tf.dtypes.cast(train_images,tf.float32)], axis=0)
        
        y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
        
        discriminator.trainable = True
        
        discriminator.train_on_batch(X_fake_vs_real, y1)
        
        y2 = tf.constant([[1.]] * batch_size)
        
        discriminator.trainable = False
        
        GAN.train_on_batch(captions, y2)

        train_captions.clear()
        train_images.clear()
        print(len(train_captions),len(train_images))
        batch_no += 1
        t2 = t.time()
        print(t2-t1)

Found GPU at: /device:GPU:0
Epoch 1/1
(100, 47)
bacth number : 1
0 0
2.2793710231781006
(100, 47)
bacth number : 2
0 0
3.0291318893432617
(100, 47)
bacth number : 3
0 0
3.835653781890869
(100, 47)
bacth number : 4
0 0
4.656109571456909
(100, 47)
bacth number : 5
0 0
5.4295055866241455
(100, 47)
bacth number : 6
0 0
6.234215974807739
(100, 47)
bacth number : 7
0 0
7.035911321640015
(100, 47)
bacth number : 8
0 0
7.861117362976074
(100, 47)
bacth number : 9
0 0
8.721991539001465
(100, 47)
bacth number : 10
0 0
9.491224765777588
(100, 47)
bacth number : 11
0 0
10.313236951828003
(100, 47)
bacth number : 12
0 0
11.088560581207275
(100, 47)
bacth number : 13
0 0
11.910758018493652
(100, 47)
bacth number : 14
0 0
12.702704668045044
(100, 47)
bacth number : 15
0 0
13.522661685943604
(100, 47)
bacth number : 16
0 0
14.340317010879517
(100, 47)
bacth number : 17
0 0
15.165721893310547
(100, 47)
bacth number : 18
0 0
15.974199533462524
(100, 47)
bacth number : 19
0 0
16.868754625320435
(100, 47)

KeyboardInterrupt: ignored

In [33]:
plt.imshow(train_images[1].shape(28,28))
print(len(train_images))
print(decode_sequence([train_captions[123]]))

TypeError: ignored